# Predict onset of diabetes using Pima Indians dataset

- Source 1: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
- Source 2: https://machinelearningmastery.com/5-step-life-cycle-neural-network-models-keras/
- Dataset: https://archive.ics.uci.edu/ml/datasets/diabetes

# 1. Setup

In [1]:
# ! conda install keras -y

In [2]:
# ! conda install tensorflow -y

In [89]:
# imports
from numpy import loadtxt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

In [90]:
# data source
# url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv'
url = 'data/pima-indians-diabetes.data.csv'

In [91]:
# load the dataset
dataset = loadtxt(url, delimiter=',')

In [92]:
# without numpy
df = pd.read_csv(url, header=None)
df.columns = ['pregnancies', 'plasma glucose', 'blood pressure', 
              'skin fold', 'insulin', 'BMI', 'pedigree', 'age', 'onset']
df.shape

(768, 9)

In [93]:
# what the dataset looks like
df.head()

,pregnancies,plasma glucose,blood pressure,skin fold,insulin,BMI,pedigree,age,onset
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [94]:
# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]

In [95]:
# examine the target
y[:5]

array([1., 0., 1., 0., 1.])

In [96]:
# split into 67% for train and 33% for test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# 2. Define Keras Model

In [97]:
# keras has two APIs: Sequential and Functional.
model = Sequential()

In [98]:
# input layer
model.add(Dense(12, input_dim=8, activation='relu'))

In [99]:
# hidden layer
model.add(Dense(16, activation='relu'))

In [100]:
# output layer
model.add(Dense(1, activation='sigmoid'))

# 3. Compile Keras Model

In [101]:
# compile the keras model
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy']
             )

# 4. Fit the model

In [102]:
# fit the keras model on the dataset

model.fit(X_train, 
          y_train, 
          validation_data=(X_test,y_test), 
          epochs=10, 
          batch_size=10)

Epoch 1/10
52/52 [==============================] - 1s 6ms/step - loss: 4.2347 - accuracy: 0.4988 - val_loss: 1.6647 - val_accuracy: 0.4724
Epoch 2/10
52/52 [==============================] - 0s 3ms/step - loss: 1.4408 - accuracy: 0.4457 - val_loss: 1.0057 - val_accuracy: 0.6102
Epoch 3/10
52/52 [==============================] - 0s 2ms/step - loss: 0.9125 - accuracy: 0.5584 - val_loss: 1.1143 - val_accuracy: 0.6614
Epoch 4/10
52/52 [==============================] - 0s 2ms/step - loss: 0.7813 - accuracy: 0.6591 - val_loss: 0.8863 - val_accuracy: 0.6654
Epoch 5/10
52/52 [==============================] - 0s 2ms/step - loss: 0.8214 - accuracy: 0.6352 - val_loss: 0.8755 - val_accuracy: 0.6654
Epoch 6/10
52/52 [==============================] - 0s 2ms/step - loss: 0.7861 - accuracy: 0.6240 - val_loss: 0.9198 - val_accuracy: 0.6654
Epoch 7/10
52/52 [==============================] - 0s 2ms/step - loss: 0.7564 - accuracy: 0.6470 - val_loss: 0.6884 - val_accuracy: 0.6811
Epoch 8/10
52/52 [==

# 5. Make Predictions


In [103]:
# make probability predictions with the model
y_probs = model.predict(X_test)
# round predictions 
rounded = [round(x[0]) for x in y_probs]
rounded[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]

In [104]:
# sample 10 of those
import numpy as np
probs=y_probs.reshape(-1)
np.random.choice(probs, size=10)

array([0.14551416, 0.30229127, 0.14671302, 0.34060773, 0.69809765,
       0.20939618, 0.12789881, 0.40560418, 0.00552759, 0.38638797],
      dtype=float32)

In [105]:
# make class predictions with the model
y_preds = (model.predict(X_test) > 0.5).astype(int)
print(y_preds[:10].tolist())
print(y_test[:10])

[[0], [0], [0], [0], [0], [0], [0], [0], [1], [1]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


# 6. Evaluate

#### Important note:
For a lot of reasons, the evaluation metrics will vary dramatically with each run of the model. A primary reason for this is the small size of the dataset.

In [106]:
# evaluate the keras model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

8/8 [==============================] - 0s 1ms/step - loss: 0.6713 - accuracy: 0.6614
Accuracy: 66.14


#### How to read the classification report
- Accuracy: The percentage of predictions that were accurate.
- Precision: Percentage of correct positive predictions relative to total positive predictions.
- Recall: Percentage of correct positive predictions relative to total actual positives.
- F1 Score: A weighted harmonic mean of precision and recall.
- Support: The number of occurrences of each class in y_test (i.e., how many observations belonged to each class in the test dataset).

In [107]:
# Evaluate the model
from sklearn import metrics
print(metrics.classification_report(y_test, y_preds))

              precision    recall  f1-score   support

         0.0       0.70      0.87      0.77       168
         1.0       0.50      0.26      0.34        86

    accuracy                           0.66       254
   macro avg       0.60      0.56      0.56       254
weighted avg       0.63      0.66      0.63       254



# 7. Save the model

In [20]:
model.save("diabetes-model-1.h5")

In [21]:
# load model
from keras.models import load_model
model2 = load_model("diabetes-model-1.h5")

# 8. Alternative: Categorical target as 2 variables

In [80]:
# split into input (X) and output (y) variables
from keras.utils import to_categorical
X = dataset[:,0:8]
y = to_categorical(dataset[:,8])

In [81]:
# examine the target & compare to the previous structure
print('before: \n', dataset[:,8][:5])
print('after: \n',  y[:5])

before: 
 [1. 0. 1. 0. 1.]
after: 
 [[0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]]


In [82]:
# split into 67% for train and 33% for test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [83]:
# build the model and fit to the data
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(16, activation='relu'))

In [84]:
# let's restructure the output layer
# previously we did this: 
#model.add(Dense(1, activation='sigmoid')) 
model.add(Dense(2, activation='softmax'))  
# notice the difference in nodes and activation function

In [85]:
# compile and fit
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy']
             )
model.fit(X_train, 
          y_train, 
          validation_data=(X_test,y_test), 
          epochs=10, 
          batch_size=10)

Epoch 1/10
52/52 [==============================] - 1s 6ms/step - loss: 5.9707 - accuracy: 0.6671 - val_loss: 1.4234 - val_accuracy: 0.6850
Epoch 2/10
52/52 [==============================] - 0s 2ms/step - loss: 1.2447 - accuracy: 0.6791 - val_loss: 1.1873 - val_accuracy: 0.6929
Epoch 3/10
52/52 [==============================] - 0s 2ms/step - loss: 1.2187 - accuracy: 0.6792 - val_loss: 1.1164 - val_accuracy: 0.5984
Epoch 4/10
52/52 [==============================] - 0s 2ms/step - loss: 0.9442 - accuracy: 0.6880 - val_loss: 1.0911 - val_accuracy: 0.6024
Epoch 5/10
52/52 [==============================] - 0s 2ms/step - loss: 1.0049 - accuracy: 0.6185 - val_loss: 0.9546 - val_accuracy: 0.6890
Epoch 6/10
52/52 [==============================] - 0s 2ms/step - loss: 0.9176 - accuracy: 0.6296 - val_loss: 0.8831 - val_accuracy: 0.6535
Epoch 7/10
52/52 [==============================] - 0s 3ms/step - loss: 0.8648 - accuracy: 0.6449 - val_loss: 0.8324 - val_accuracy: 0.6969
Epoch 8/10
52/52 [==

In [86]:
# examine the predictions
y_probs = model.predict(X_test)
print(y_probs[:5])

y_preds = (model.predict(X_test) > 0.5).astype(int)
print(y_preds[:5])

[[0.43301982 0.5669802 ]
 [0.9786235  0.02137655]
 [0.957305   0.04269496]
 [0.9732276  0.02677243]
 [0.9708293  0.0291707 ]]
[[0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]]


In [87]:
# reformat the outcomes to a single value (this is optional)
y_test=[y[1] for y in y_test]
y_preds=[pred[1] for pred in y_preds]

In [88]:
# evaluate
print(metrics.classification_report(y_test, y_preds))

              precision    recall  f1-score   support

         0.0       0.74      0.85      0.79       168
         1.0       0.57      0.41      0.48        86

    accuracy                           0.70       254
   macro avg       0.65      0.63      0.63       254
weighted avg       0.68      0.70      0.68       254

